# PyJAMAS notebook 20240612_095609

We start by importing the packages necessary to run and plot the analysis. We also create lists that define what we will be plotting.

In [ ]:
import ipywidgets as widgets
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import numpy
import pandas as pd
import seaborn as sns

In [ ]:
TIME_PLOTS = ['area (μm²)', 'perimeter (μm)', 'circularity (dimensionless)', 'raw pixel values interior', 'raw pixel values perimeter', 'image mean', 'image mode', 'normalized pixel values interior', 'normalized pixel values perimeter', 'std interior', 'std perimeter', 'normalized std interior', 'normalized std perimeter', 'heterogeneity interior', 'heterogeneity perimeter', 'total normalized pixel values interior', 'total normalized pixel values perimeter', 'scaled normalized pixel values interior', 'scaled normalized pixel values perimeter', 'area (%)', 'perimeter (%)', 'circularity (%)', 'raw pixel values interior (%)', 'raw pixel values perimeter (%)']
BOX_PLOTS = ['closure rate constant - area (1/min)', 'closure rate constant - perimeter (1/min)', 'area (% change)', 'perimeter (% change)', 'circularity (% change)', 'normalized pixel values interior (% change)', 'normalized pixel values perimeter (% change)', 'total normalized pixel values interior (% change)', 'total normalized pixel values perimeter (% change)', 'heterogeneity interior (% change)', 'heterogeneity perimeter (% change)']

Run the analysis (uncomment the code in this cell; otherwise see below to load analysis results from disk):

In [ ]:
# import sys
# sys.path.extend(['/Users/rodrigo/src/pyjamas_dev'])
# from pyjamas.pjscore import PyJAMAS

# a = PyJAMAS()

# parameters = {'folders': ['/Users/rodrigo/src/pyjamas_dev/pyjamas/tests/unit/fixtures/batch_measure/water_1', '/Users/rodrigo/src/pyjamas_dev/pyjamas/tests/unit/fixtures/batch_measure/water_2', '/Users/rodrigo/src/pyjamas_dev/pyjamas/tests/unit/fixtures/batch_measure/juice'], 'analyze_flag': True, 'analysis_filename_appendix': '_analysis', 'analysis_extension': '.csv', 'save_results': True, 'script_filename_appendix': '_analysis_script.ipynb', 'results_folder': '/Users/rodrigo/src/pyjamas_dev/pyjamas/tests/unit/fixtures/batch_measure/results', 'intensity_flag': True, 'image_extension': '.tif', 'normalize_intensity_flag': 0, 't_res': 30.0, 'xy_res': 0.178, 'index_time_zero': 2, 'plot_flag': False, 'names': ['water 1', 'water 2', 'juice'], 'err_style_value': 'band', 'plot_style_value': 'box', 'brush_sz': 3}

# a.batch.cbMeasureBatch(parameters)

Or load analysis results from disk:

In [ ]:
all_data = pd.read_csv('/Users/rodrigo/src/pyjamas_dev/pyjamas/tests/unit/fixtures/batch_measure/results/20240612_095609_analysis.csv')

Plot results: first, set up plots.

In [ ]:
sns.set()
sns.set_style("white")
plt.rcParams['font.size'] = 14
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['figure.figsize'] = [8, 6]
# plt.xkcd()  # xkcd plotting style :)

n_groups: int = len(numpy.unique(all_data['experimental group']))
a_palette = sns.color_palette('bright', n_groups)

## Means plots comparing two groups.
You may use *plt.savefig("file_name.ext")* after the code that creates a figure to save the figure. *ext* can be png, svg, etc ...

In [ ]:
@widgets.interact(metric=TIME_PLOTS)
def mean_time_plots(metric):
    ax = sns.lineplot(x='time (min)', y=metric, hue='experimental group', data=all_data, errorbar='se', err_style='band', lw=3, palette=a_palette)
    handles, _ = ax.get_legend_handles_labels()
    ax.legend(handles=handles, labels=['water 1', 'water 2', 'juice'], frameon=False)
    sns.despine()
    return

## Plot individual curves for each of the experiments.
You may use *plt.savefig("file_name.ext")* after the code that creates a figure to save the figure. *ext* can be png, svg, etc ...

In [ ]:
@widgets.interact(metric=TIME_PLOTS)
def individual_time_plots(metric):
    ax = sns.lineplot(x='time (min)', y=metric, hue='experimental group', data=all_data, estimator=None, units='experiment index', lw=3, legend='full', palette=a_palette)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles=handles[0:], labels=labels[0:], frameon=False)
    sns.despine()
    
    for a_group in ['water 1', 'water 2', 'juice']:
        plt.figure()
        thedata = all_data[all_data['experimental group'] == a_group]
        n_exp: int = len(numpy.unique(thedata['experiment index']).astype(int))
        ax = sns.lineplot(x='time (min)', y=metric, hue='experiment index', data=thedata, estimator=None, units='experiment index', lw=3, palette=sns.color_palette('husl', n_exp))
        ax.legend([str(x) for x in range(n_exp)], frameon=False, title=a_group)
        sns.despine()
    
    return

## Box plots for summary metrics.
You may use *plt.savefig("file_name.ext")* after the code that creates a figure to save the figure. *ext* can be png, svg, etc ...
Also note that, if you want to look at the *initial* value of any time features, you can replace *BOX_PLOTS* with *TIME_PLOTS* below.

In [ ]:
thedata = all_data.groupby(['experimental group', 'experiment index'], as_index=False, sort=False).first()

In [ ]:
@widgets.interact(metric=BOX_PLOTS)
def box_plots(metric):
    ax = sns.boxplot(x='experimental group', y=metric, data=thedata, hue='experimental group', dodge=False, whis=100000, palette=a_palette)
    handles, _ = ax.get_legend_handles_labels()
    ax.legend(handles=handles, frameon=False)
    ax = sns.stripplot(x='experimental group', y=metric, data=thedata, color='k', alpha=0.75, size=6, dodge=False, jitter=0.05)
    sns.despine()
    return